# Hollow Passage

### Install required libraries

In [1]:
!pip install -r requirements.txt

### Import Text Adventures

In [2]:
from text_adventure_games import (
    games, parsing, actions, things, blocks, viz
)

### Locations, Character, Layout, and Items

In [ ]:
from text_adventure_games import things, games, actions

#Locations
foyer = things.Location("foyer", "A creaking foyer with peeling wallpaper and a heavy front door.")
landing = things.Location("landing", "A narrow landing with stairs leading down.")
basement = things.Location("basement", "Cold air rises from the pitch-black basement.")

#Connections
foyer.add_connection("north", landing, "You creep toward the stairs.")
landing.add_connection("south", foyer, "You return to the foyer.")
landing.add_connection("down", basement, "You descend into the basement.")
basement.add_connection("up", landing, "You climb back to the landing.")

#Items: candle (lightable), key, front door (locked + openable)
candle = things.Item("candle", "a slender red candle", "Wax drips line its sides.")
candle.set_property("is_lightable", True)
candle.set_property("is_lit", False)

key = things.Item("key", "an iron key", "Its teeth are rough and old.")

front_door = things.Item("front door", "the front door", "A heavy, iron-banded door with a deadbolt.")
front_door.set_property("is_locked", True)
front_door.set_property("is_open", False)

#Place items (player can pick up candle; key is in basement)
foyer.add_item(candle)
foyer.add_item(front_door)
basement.add_item(key)

#Player
player = things.Character(
    name="you",
    description="A lost soul who badly wants to leave.",
    persona="I need to get out."
)

#Add player to foyer
foyer.add_character(player)

### Actions

In [4]:
from text_adventure_games import actions

class Light_Candle(actions.Action):
    ACTION_NAME = "light candle"
    ACTION_DESCRIPTION = "Light a candle if you have it."
    ACTION_ALIASES = ["light"]

    def __init__(self, game, command):
        super().__init__(game)
        self.character = self.parser.get_character(command)
        inv = self.character.inventory if self.character else {}
        self.candle = self.parser.match_item("candle", inv)
        if not self.candle and self.character:
            self.candle = self.parser.match_item("candle", self.character.location.items)

    def check_preconditions(self) -> bool:
        if not self.was_matched(self.character, "No character found."):
            return False
        if not self.was_matched(self.candle, "You don't have a candle to light."):
            return False
        if not self.candle.get_property("is_lightable"):
            self.parser.fail("That can't be lit.")
            return False
        if self.candle.get_property("is_lit"):
            self.parser.fail("The candle is already lit.")
            return False
        return True

    def apply_effects(self):
        self.candle.set_property("is_lit", True)
        self.parser.ok("You strike a spark. The candle flickers to life.")

class Unlock_Front_Door(actions.Action):
    ACTION_NAME = "unlock door"
    ACTION_DESCRIPTION = "Unlock the front door if you have the key."
    ACTION_ALIASES = ["unlock"]

    def __init__(self, game, command):
        super().__init__(game)
        self.character = self.parser.get_character(command)
        self.location = self.character.location if self.character else None
        self.door = self.parser.match_item("front door", self.location.items if self.location else {})
        inv = self.character.inventory if self.character else {}
        self.key = self.parser.match_item("key", inv)

    def check_preconditions(self) -> bool:
        if not self.was_matched(self.character, "No character found."):
            return False
        if not self.was_matched(self.door, "There is no front door here."):
            return False
        if not self.was_matched(self.key, "You don't have the key."):
            return False
        if not self.door.get_property("is_locked"):
            self.parser.fail("The door is already unlocked.")
            return False
        return True

    def apply_effects(self):
        self.door.set_property("is_locked", False)
        self.parser.ok("You turn the iron key. The deadbolt thunks open.")

class Open_Front_Door(actions.Action):
    ACTION_NAME = "open door"
    ACTION_DESCRIPTION = "Open the front door and escape (if it's unlocked)."
    ACTION_ALIASES = ["open"]

    def __init__(self, game, command):
        super().__init__(game)
        self.character = self.parser.get_character(command)
        self.location = self.character.location if self.character else None
        self.door = self.parser.match_item("front door", self.location.items if self.location else {})

    def check_preconditions(self) -> bool:
        if not self.was_matched(self.character, "No character found."):
            return False
        if not self.was_matched(self.door, "There is no front door here."):
            return False
        if self.door.get_property("is_locked"):
            self.parser.fail("It won't budge. The deadbolt is still locked.")
            return False
        if self.door.get_property("is_open"):
            self.parser.fail("The door already stands open.")
            return False
        return True

    def apply_effects(self):
        self.door.set_property("is_open", True)
        self.parser.ok("You heave the door inward. Night air floods the foyer... You escape!")
        self.game.game_over = True
        self.game.game_over_description = "You escaped the haunted house. THE END."

### Win and loss states

In [5]:
class TinyHorror(games.Game):
    def __init__(self, start_at, player, characters=None, custom_actions=None):
        super().__init__(start_at, player, characters=characters, custom_actions=custom_actions)

    def is_won(self) -> bool:
        return False  # win is set when opening the front door

    def is_game_over(self) -> bool:
        if self.game_over or self.player.get_property("is_dead"):
            return super().is_game_over()

        at_basement = self.player.location == basement
        has_lit_light = any(
            itm.get_property("is_lightable") and itm.get_property("is_lit")
            for itm in self.player.inventory.values()
        )
        if at_basement and not has_lit_light:
            self.player.set_property("is_dead", True)
            self.game_over_description = "You stumble in the black and something cold closes around your throat. THE END."
            return True
        return super().is_game_over()

In [6]:
# === Instantiate Tiny Horror and register actions ===
custom_actions = [Light_Candle, Unlock_Front_Door, Open_Front_Door]
game = TinyHorror(foyer, player, characters=[player], custom_actions=custom_actions)
print(game.describe())

A creaking foyer with peeling wallpaper and a heavy front door.
Exits:
North to landing

You see:
 * a stubby tallow candle
 * the front door




In [ ]:
game.game_loop()

A creaking foyer with peeling wallpaper and a heavy front door.
Exits:
North to landing

You see:
 * a stubby tallow candle
 * the front door


you got the candle.
I'm not sure what you want to do.
I'm not sure what you want to do.
